In [1]:
#default_exp github_crawling

In [94]:
#export
import pandas as pd
import requests
import json
from operator import itemgetter
import tqdm

import os
import tqdm
import glob
from git.repo.base import Repo
import pathlib
from github_search import paperswithcode_tasks

import itertools
import codecs
import concurrent.futures

In [3]:
%cd ..

/home/kuba/Projects/github_search


# Crawling github

Getting Python files from github repos

Before running that code put your github auth token in data/token.txt. Doing this will increase rate limit from 60 to 5000 calls per hour.

`_get_tree` gets file tree from github repository

`_get_python_files` retrieves Python files from a given repository: it returns tuples consisting of

owner, repo_name, file path, file content, hash of file content


`get_python_files_df` puts information described above into dataframe with appropriate column names 

In [71]:
#export

token = open('data/token.txt', 'r').read().strip()


def _get_tree(owner, repo_name):
    url_template = 'https://api.github.com/repos/{}/{}/git/trees/master?recursive=1'
    headers = {'Authorization': 'token ' + token}
    url = url_template.format(owner, repo_name)
    return requests.get(url, headers=headers)


def _get_python_files(owner, repo_name):
    try:
        files = json.loads(_get_tree(owner, repo_name).text)['tree']
        for maybe_file in files:
            path = maybe_file['path']
            extension = path.split('.')[-1]
            is_valid_file = extension == 'py'# or os.path.basename(path) == 'README.md' 
            if maybe_file['type'] == 'blob' and is_valid_file:
                path = maybe_file['path']
                raw_file_url_template = 'https://raw.githubusercontent.com/{}/{}/master/{}'
                raw_file_url = raw_file_url_template.format(owner, repo_name, path)
                yield owner, repo_name, path, requests.get(raw_file_url).text, maybe_file['sha']
            
    except (KeyError, json.JSONDecodeError) as e:
        print('failed for {}/{}'.format(owner, repo_name))
        print(type(e))
        print(e)
        return
        yield

In [72]:
example_owner = 'lambdaofgod'
example_repo = 'mlutil'

Getting python files should return empty list when ran on nonexisting repository

In [73]:
assert list(_get_python_files('lambdaofgod', 'examples_counterexamples')) == []

failed for lambdaofgod/examples_counterexamples
<class 'KeyError'>
'tree'


Getting python files should return correct number of files for an existing repository 

In [79]:
python_file_tuples = list(_get_python_files('lambdaofgod', 'mlutil'))
assert len(python_file_tuples) == 27 

In [80]:
#export


def _make_python_files_df(file_tuples):
    if len(file_tuples) > 0:
        df = pd.DataFrame.from_records(file_tuples)
        df.columns = ['owner', 'repo_name', 'file_path', 'content', 'sha']
        return df
    else:
        return pd.DataFrame({})


def get_python_files_df(owner, repo_name):
    file_tuples = list(_get_python_files(owner, repo_name))
    return _make_python_files_df(file_tuples)

In [81]:
df_cols = list(_make_python_files_df(python_file_tuples).columns)
assert df_cols == ['owner', 'repo_name', 'file_path', 'content', 'sha']

## Brute force - clone repos

In [82]:
!mkdir -p repos

In [123]:
#export

def clone_repo(destination_prefix, repo_name):
    dest_path = os.path.join(destination_prefix, repo_name)
    if not os.path.exists(dest_path):
        Repo.clone_from("https://github.com/{}".format(repo_name), dest_path)
        for p in pathlib.Path(dest_path).rglob("*"):
            if os.path.isfile(p) and not (str(p).endswith(".py") or str(p).endswith(".md")):
                os.remove(p)
                
def clone_repos(repos, max_workers=2, destination_prefix="repos"):
    def _clone_repo(repo_name):
        try:
            clone_repo(destination_prefix, repo_name)
        except Exception as e:
            print(e)
    results = list(tqdm.tqdm(map(_clone_repo, repos), total=len(repos)))

In [124]:
def get_python_files_generator_from_directory(dir_path):
    for p in pathlib.Path(dir_path).rglob("*.py"):
        try:
            with codecs.open(p, 'r', encoding='utf-8') as f:
                path = str(p)
                contents = f.read().strip().encode('utf-8').decode('utf-8')
                if len(contents) > 0:
                    yield path, contents
        except (FileNotFoundError, IsADirectoryError, UnicodeDecodeError):
            pass
            
def make_python_files_df_from_directory(dir_path):
    repo_name = "/".join(dir_path.split("/")[-2:])
    paths_with_contents = list(zip(*list(get_python_files_generator_from_directory(dir_path))))
    if len(paths_with_contents) == 2:
        paths, contents = zip(*list(get_python_files_generator_from_directory(dir_path)))
        df = pd.DataFrame({"content": contents, "path": paths})
        df['path'] = df['path'].str.replace(dir_path + "/", "")
        df['repo_name'] = repo_name
        return df

In [125]:
def check_if_python_files_exist(dir_prefix, repo):
    maybe_python_files = itertools.islice(pathlib.Path("repos/" + repo).rglob("*.py"), 1)
    return len(list(maybe_python_files)) > 0

In [120]:
paperswithcode_df, all_papers_df = paperswithcode_tasks.get_paperswithcode_dfs()

Exception ignored in: <function Repo.__del__ at 0x7f1714a09ee0>
Traceback (most recent call last):
  File "/home/kuba/.local/lib/python3.8/site-packages/git/repo/base.py", line 218, in __del__
    try:
KeyboardInterrupt: 


In [127]:
!ls repos/

In [128]:
clone_repos(paperswithcode_df['repo'].iloc[:paperswithcode_df.shape[0] // 2])

  0%|          | 151/46292 [06:46<20:24:13,  1.59s/it]

Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v https://github.com/GartontT/CHIMERA repos/GartontT/CHIMERA
  stderr: 'Cloning into 'repos/GartontT/CHIMERA'...
fatal: could not read Username for 'https://github.com': No such device or address
'


  0%|          | 218/46292 [08:40<9:47:17,  1.31it/s] 

Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v https://github.com/ladadidadida/Self-Attention-GAN repos/ladadidadida/Self-Attention-GAN
  stderr: 'Cloning into 'repos/ladadidadida/Self-Attention-GAN'...
fatal: could not read Username for 'https://github.com': No such device or address
'


  1%|          | 340/46292 [13:16<14:50:46,  1.16s/it]

...

100%|█████████▉| 46196/46292 [30:08:47<01:21,  1.18it/s]

Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v https://github.com/eliceio/vocal-style-transfer repos/eliceio/vocal-style-transfer
  stderr: 'Cloning into 'repos/eliceio/vocal-style-transfer'...
fatal: could not read Username for 'https://github.com': No such device or address
'


100%|█████████▉| 46235/46292 [30:10:35<00:59,  1.05s/it]

Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v https://github.com/alibaba/EasyTransfer/tree/master/scripts/fashion_bert repos/alibaba/EasyTransfer/tree/master/scripts/fashion_bert
  stderr: 'Cloning into 'repos/alibaba/EasyTransfer/tree/master/scripts/fashion_bert'...
fatal: repository 'https://github.com/alibaba/EasyTransfer/tree/master/scripts/fashion_bert/' not found
'


100%|██████████| 46292/46292 [30:13:20<00:00,  2.35s/it]


In [129]:
def prepare_python_files_df(repos):
    paperswithcode_df, all_papers_df = paperswithcode_tasks.get_paperswithcode_dfs()
    repo_files = pd.concat([
        make_python_files_df_from_directory("repos/" + repo)
        for repo in tqdm.tqdm(repos)
        if check_if_python_files_exist("repos", repo)
    ])
    return repo_files

In [130]:
python_files_df = prepare_python_files_df(paperswithcode_df['repo'])

100%|██████████| 92585/92585 [03:12<00:00, 479.76it/s]  


In [131]:
python_files_df['content'] =  python_files_df['content'].str.replace('\r', '\n')

In [133]:
python_files_df.head()

,content,path,repo_name
0,#!/usr/bin/env python3\nimport subprocess\nimp...,run-tests.py,trangvu/ape-npi
1,import pickle\n\nimport tensorflow as tf\nimpo...,translate/import_graph.py,trangvu/ape-npi
2,import tensorflow as tf\nimport math\nfrom ten...,translate/models.py,trangvu/ape-npi
3,import tensorflow as tf\nfrom tensorflow.contr...,translate/conv_lstm.py,trangvu/ape-npi
4,import tensorflow as tf\nfrom translate import...,translate/beam_search.py,trangvu/ape-npi


In [163]:
len(python_files_df['repo_name'].unique())

27008

In [164]:
len(python_files_df['repo_name'])

1806684

In [156]:
(python_files_df['repo_name'].value_counts().cumsum() / len(python_files_df)).iloc[:25]

google-research/google-research                      0.145302
tensorflow/models                                    0.243455
osmr/imgclsmob                                       0.289806
huggingface/transformers                             0.322116
OscarJHernandez/qc_mentorship_project                0.333109
pytorch/fairseq                                      0.342963
open-mmlab/mmsegmentation                            0.352375
joe-prog/https-github.com-facebookresearch-ParlAI    0.358086
TQRG/BugSwarm                                        0.362463
tensorflow/tensor2tensor                             0.366725
open-mmlab/mmpose                                    0.370496
PaddlePaddle/Research                                0.374107
rwightman/pytorch-image-models                       0.377490
Daikenan/LTMU                                        0.380106
pytorch/pytorch                                      0.382720
benedekrozemberczki/karateclub                       0.385268
VLOGroup

In [157]:
crawled_files = pd.read_csv("data/crawled_python_files.csv")

In [158]:
scraped_files = pd.read_csv("data/scraped_python_files.csv")

In [159]:
len(crawled_files)

1402272

In [160]:
len(scraped_files)

1461081

In [161]:
len(crawled_files['repo_name'].unique())

18933

In [162]:
len(scraped_files['repo_name'].unique())

18950

In [165]:
python_files_df['repo_name'].value_counts()[:20].sum() / len(python_files_df)

0.39488255832231867

In [166]:
file_lengths = python_files_df['content'].apply(len)

In [171]:
file_lengths.describe()

count    1.806684e+06
mean     9.728450e+03
std      6.029188e+04
min      1.000000e+00
25%      1.965000e+03
50%      4.690000e+03
75%      1.027500e+04
max      3.558457e+07
Name: content, dtype: float64

In [172]:
%%time
python_files_df[file_lengths < 1e5].to_csv('data/crawled_python_files.csv', index=False, encoding='utf-8')

CPU times: user 2min 7s, sys: 6.27 s, total: 2min 13s
Wall time: 2min 13s


In [77]:
python_files_df[file_lengths < 1e5].shape

(1402087, 3)

In [75]:
(file_lengths < 1e5).sum()

1402087

In [43]:
len(python_files_df.iloc[1281222]['content']

9526

In [50]:
(file_lengths > 1e6).mean()

0.00017958915962920871

In [44]:
s = """# -*- coding: utf-8 -*-\n\n######################## IMPORTING / GOOGLE DRIVE INTEGRATION ##############################\nimport torch\nimport argparse\nimport copy\nimport cv2\nimport json\nimport numpy as np\nimport os\nimport pandas as pd\nimport pdb\nimport pickle\nimport random\nimport scipy.ndimage\nimport shutil\nimport sklearn.preprocessing\nimport sys\n#import tensorflow as tf\nimport time\nimport torch\nimport torch.nn as nn\nimport torch.nn.functional as F\nimport torch.optim as optim\nimport torchvision\nfrom PIL import Image\nfrom itertools import product\nfrom matplotlib import pyplot as plt\nimport numpy as np\nimport pickle\nimport json\nimport pandas as pd\n#import tensorflow as tf\nfrom torchvision import datasets, transforms\nfrom torch.utils.tensorboard import SummaryWriter\nimport argparse\nimport sys\nimport pdb\nimport sklearn\nimport yaml\n\nfrom shutil import copy2\nfrom torch.autograd import Variable\nfrom torch.optim import lr_scheduler\nfrom torch.utils.data import Dataset, DataLoader\nfrom torch.utils.tensorboard import SummaryWriter\nfrom torchvision import datasets, transforms, models\nfrom yaml import load, dump\n\nfrom functools import partial\nimport signal\nimport time\nimport sys\nimport numpy as np\n\n################################ SET UP PATHS ##############################################\n# See README.md for more details\n\nmount_filepath = """" # Put the path to the root directory of your project here\nmodel_name_prefix = f""vae_trip_vgg""\n\n################################ SETTING UP PROGRAM ARGUMENTS ##############################\n\nIS_RUNNING_NOTEBOOK = False  # ** NOTE: PLEASE DO NOT FORGET TO CHANGE THIS FLAG WHEN YOU DEPLOY **\n\nexit_code = None\n# The safe_exit function works by only registering a flag that is\n# monitored in the actual training loop\n#\n# In addition, this is the central authority on the exit code for this program \n# and will be used for the actual exit code at the end\ndef safe_exit(signal_num, ec=0, total_iters=0, exp_name = """"):\n    print(""Caught a signal"", signal_num)\n    sys.stdout.flush()\n    global exit_code\n    exit_code = signal_num\n\nstart_time = time.time()\n\n# Catch the signal and report to safe_exit\nsignal.signal(signal.SIGTERM, safe_exit)\n\ndevice = torch.device(""cuda"")\n\nparser = argparse.ArgumentParser()\nparser.add_argument(""yaml_config"", help = ""No config file specified. Please specify a yaml file."")\nargs = parser.parse_args()\n\nconfig_file = open(args.yaml_config, 'r')\nconfig_args = yaml.load(config_file)\n\n\nLATENT_SIZE = config_args['LATENT_SIZE']\nLATENT_ARTIST_SIZE = config_args['LATENT_ARTIST_SIZE']\n\nPERCEP_RATIO = config_args['PERCEP_RATIO']\nKLD_RATIO = config_args['KLD_RATIO']\nTRIP_RATIO = config_args['TRIP_RATIO']\nRECON_RATIO = config_args['RECON_RATIO']\n\nOUTLIERS_REMOVED = config_args['OUTLIERS']\n\nTRAIN_TEST_RATIO = config_args['TRAIN_TEST_SPLIT']\n\nTIMEOUT = 3600\n\n\nMASK_TYPE = ""mask""\n\n""""""# HYPERPARAMETERS""""""\n\n################################ HYPERPARAMETERS ##############################\n\n# Base VAE Hyperparameters\nTRIPLET_ALPHA = 1.0\n#LATENT_ARTIST_WEIGHT = 1.0 * NON_VAE_WEIGHT\nRECONSTRUCTION_WEIGHT = RECON_RATIO\nPERCEP_LOSS_WEIGHT = PERCEP_RATIO\nVAE_DIVERGENCE_WEIGHT = KLD_RATIO\nTRIPLET_LOSS_WEIGHT = TRIP_RATIO\n\n# Loss Thresholds\nMAX_VAE_LOSS_THRESHOLD = 200\nMAX_LATENT_ARTIST_LOSS_THRESHOLD = 10\n\n# Random sampling hyperparameters\nSAME_ARTIST_PROBABILITY_THRESHOLD = 0.7\n\n# Train Test Split\nTRAIN_TEST_SPLIT = TRAIN_TEST_RATIO\n\ninterval = 1 # Interval for which to print the loss\nnum_batches_per_epoch = 478\nepochs = num_batches_per_epoch * 500\nepoch = 0 # Start epoch to run from (gets automatically set in the checkpointing process)\n\n\n################################ SETTING UP DIRECTORY STRUCTURE ##############################\n\n## TODO: Update this with a better name? Let's stick with 1024 parameters and 512 for the artist latent vector\n\n# hyperparameter_name = f""kld_{VAE_DIVERGENCE_WEIGHT}_percep_{PERCEP_LOSS_WEIGHT}_recon_{RECONSTRUCTION_WEIGHT}_trip_{TRIPLET_LOSS_WEIGHT}""\n#hyperparameter_name = f""nonvaeweight_{NON_VAE_WEIGHT}_artist_{LATENT_ARTIST_SIZE}_latent_{LATENT_SIZE}""\n\nhyperparameter_name = f""TRIPLET_{TRIPLET_LOSS_WEIGHT}"" \n\nmodel_name = f""{model_name_prefix}_{hyperparameter_name}"" # Name of the model generated from the program arguments\n\nwriter = SummaryWriter(f'./tensorboard/{model_name}') # Tensorboard writer\n# kld_writer = SummaryWriter(f'./tensorboard/{model_name}/kld_loss') # Tensorboard writer\n# recon_writer = SummaryWriter(f'./tensorboard/{model_name}/recon_loss') # Tensorboard writer\n# artist_writer = SummaryWriter(f'./tensorboard/{model_name}/artist_loss') # Tensorboard writer\ntriplet_writer = SummaryWriter(f'./tensorboard/{model_name}/triplet_loss') # Tensorboard writer\n# perceptual_writer = SummaryWriter(f'./tensorboard/{model_name}/perceptual_loss') # Tensorboard writer\n\nprefix = ""padded_"" # Prefix for all the files being saved\ncontinue_training = True # Specifies whether you want to continuing training from the previous epoch. Honestly don't know why you'd set it to false but you can if you want.\nmodels_folder = f""{mount_filepath}/saved_models/{model_name}/models""\n\nmodel_save_root = mount_filepath + '/saved_models'\nresults_folder = f""{model_save_root}/{model_name}/images""\nplot_folder = f""{model_save_root}/{model_name}/plots""\nlog_folder = f""{model_save_root}/{model_name}/logs""\n\nload_model_folder = f""{model_save_root}/{model_name}/models""\n\nos.makedirs(f""{model_save_root}/{model_name}"", exist_ok=True)\nos.makedirs(results_folder, exist_ok=True)\nos.makedirs(plot_folder, exist_ok=True)\n\nLATENT_PREFIX = ""padded_""\n\n#DISCRIMINATOR_PREFIX = ""discriminator"" # DONT CHANGE THIS\n\n#LATENT_ARTIST_MODEL_NAME = f""{DISCRIMINATOR_PREFIX}_{model_name_prefix}_{hyperparameter_name}""\n#os.makedirs(f""{model_save_root}/{LATENT_ARTIST_MODEL_NAME}"", exist_ok=True)\n#os.makedirs(f""{model_save_root}/{LATENT_ARTIST_MODEL_NAME}/images"", exist_ok=True)\n#os.makedirs(f""{model_save_root}/{LATENT_ARTIST_MODEL_NAME}/plots"", exist_ok=True)\n#os.makedirs(f""{model_save_root}/{LATENT_ARTIST_MODEL_NAME}/logs"", exist_ok=True)\n#os.makedirs(f""{model_save_root}/{LATENT_ARTIST_MODEL_NAME}/models"", exist_ok=True)\n\n# If you're training the base VAE, you must still specify a load path for the latent artist network\n#LATENT_ARTIST_MODEL_LOAD_PATH = f""{model_save_root}/{LATENT_ARTIST_MODEL_NAME}/models""\n\n#model_name = LATENT_ARTIST_MODEL_NAME\n#prefix = LATENT_PREFIX\n#load_model_folder = LATENT_ARTIST_MODEL_LOAD_PATH\n\n""""""# SETTING UP PATHS / DIRECTORY STRUCTURE""""""\n\n################################ SETTING UP PATHS / DIRECTORY STRUCTURE ##############################\nmetadata_filepath = mount_filepath + 'FINAL_DATASET_SIZES.csv'\nimage_zip_folder = mount_filepath + 'Images.zip'\nimage_folder = mount_filepath + ""Images""\n# triplets_path = mount_filepath + ""Triplets/4700_labels_copy.json"" \ntriplets_path = mount_filepath + ""Triplets/6384_labels.json""\n\nwriter = SummaryWriter(f'./tensorboard/{model_name}') # Tensorboard writer\nprefix = ""padded_"" # Prefix for all the files being savedc\nmodels_folder = f""{mount_filepath}/saved_models/{model_name}/models""\n\nmodel_save_root = mount_filepath + '/saved_models'\nresults_folder = f""{model_save_root}/{model_name}/images""\nlog_folder = f""{model_save_root}/{model_name}/logs""\n\nos.makedirs(f""{model_save_root}/{model_name}"", exist_ok=True)\nos.makedirs(results_folder, exist_ok=True)\nos.makedirs(log_folder, exist_ok=True)\nos.makedirs(f""{model_save_root}/{model_name}/models"", exist_ok=True)\n\n""""""# ART DATASET CLASS""""""\n\n################################ ART DATASET CLASS ##############################\n\nclass ArtDataset(Dataset):\n    def __init__(self, image_folder, triplet_df, metadata_df, largest_height, largest_width, transform=None):\n        """"""\n        Args:\n            image_folder (string): Directory with all the images.\n            metadata_df (Pandas Dataframe): Dataframe containing the metadata\n            transform (Torchvision.Transform) Transform to be applied to the image data\n        """"""\n        self.transform = transform\n        self.metadata = metadata_df.reset_index()\n        self.triplets = triplet_df\n        self.image_folder = image_folder\n        self.largest_height = largest_height\n        self.largest_width = largest_width\n\n    def __len__(self):\n        return len(self.triplets.index)\n\n    def return_transformed_image(self, image_filename):\n        image = Image.open(os.path.join(self.image_folder, image_filename))\n        (width, height) = image.size\n        tensor_image = self.transform(image)\n        pad_layer = nn.ZeroPad2d((0, self.largest_width - width, 0, self.largest_height - height))\n        return pad_layer(tensor_image)\n\n    def get_random_artist_image(self, artist):\n        artist_indices = self.metadata[self.metadata['cleaned_artist'] == artist].index\n        random_artist_idx = random.choice(artist_indices)\n        return self.metadata.iloc[random_artist_idx]\n\n    def __getitem__(self, idx):\n      row = self.triplets.iloc[idx]\n\n      positive_idx = int(row['Positive']) - 1\n      negative_idx = (1 - positive_idx)\n\n      triplet_names = ['1', '2', 'anchor']\n\n      # Re-Arranges it so now it's of the form positive, negative, anchor.\n      triplet_files = [row[triplet_names[positive_idx]], row[triplet_names[negative_idx]], row[triplet_names[2]]]\n      labels = ['positive', 'negative', 'anchor']\n\n      triplet_objects = {}\n\n      for filename, label in zip(triplet_files, labels):\n        triplet_object = {}\n\n        triplet_image = {}\n        triplet_corresponding = {}\n\n        image_metadata = self.metadata.loc[self.metadata['filename'] == filename]\n        image_name = image_metadata['filename'].values[0]\n\n        triplet_image['artist'] = image_metadata['cleaned_artist'].values[0]\n        triplet_image['image'] = self.return_transformed_image(image_name)\n\n        triplet_image['normalized_midpoint'] = image_metadata['normalized_midpoint'].values[0]\n        triplet_image['normalized_start'] = image_metadata['normalized_start'].values[0]\n        triplet_image['normalized_end'] = image_metadata['normalized_end'].values[0]\n\n        triplet_image['width'] = image_metadata['width'].values[0]\n        triplet_image['height'] = image_metadata['height'].values[0]\n\n        #random_artist = self.get_random_artist_image(triplet_image['artist'])\n\n        #triplet_corresponding['image'] = self.return_transformed_image(random_artist['filename'])\n        #triplet_corresponding['filename'] = random_artist['filename']\n\n        #triplet_corresponding['normalized_midpoint'] = random_artist['normalized_midpoint']\n        #triplet_corresponding['normalized_start'] = random_artist['normalized_start']\n        #triplet_corresponding['normalized_end'] = random_artist['normalized_end']\n\n        #triplet_corresponding['width'] = random_artist['width']\n        #triplet_corresponding['height'] = random_artist['height']\n\n        #triplet_corresponding['should_mask'] = (random_artist['filename'] == image_metadata['filename'].values[0])\n\n        triplet_object['triplet'] = triplet_image\n        #triplet_object['corresponding'] = triplet_corresponding\n\n        triplet_objects[label] = triplet_object\n\n      return triplet_objects\n\n""""""# READING IN DATA FROM CSV / PREPROCESSING""""""\n\n################################ READING IN DATA FROM CSV / PREPROCESSING ##############################\n\nimport json\nimport pandas as pd\n#pdb.set_trace()\n\nmetadata_filepath = mount_filepath + 'FINAL_DATASET_SIZES.csv'\nimage_zip_folder = mount_filepath + 'Images.zip'\n\nimage_folder = mount_filepath + ""Images""\n\nmetadata_df = pd.read_csv(metadata_filepath)\nprint(metadata_df.shape)\nmetadata_df = metadata_df.fillna("""")\nartists_column = metadata_df['cleaned_artist'].str.lower()\nknown_artists = metadata_df[(artists_column != 'unsure') & (artists_column != 'anonymous')]\n\nanonymous_artists = metadata_df[(artists_column == 'unsure') ^ (artists_column == 'anonymous')]\n\n# To use all the data just set min_thresh = 0 and max_thresh = 1\nmin_thresh = 0.00\nmax_thresh = 1.00\n\n# Only keep the artists that are known\n# CURRENTLY COMMENTED SINCE WE DON'T CARE IF THE ARTISTS ARE UNKOWN!\n# metadata_df = known_artists\n\nsorted_widths = sorted(metadata_df['width'].tolist())\nsorted_heights = sorted(metadata_df['height'].tolist())\n\nif OUTLIERS_REMOVED == ""outliers"":\n  bottom_range_height = sorted_heights[max(0, int(len(sorted_heights) * min_thresh) - 1)]\n  top_range_height = sorted_heights[max(0, int(len(sorted_heights) * max_thresh) - 1)] + 1\n  bottom_range_width = sorted_widths[max(0, int(len(sorted_widths) * min_thresh) - 1)]\n  top_range_width = sorted_widths[max(0, int(len(sorted_widths) * max_thresh) - 1)]\nelse:\n  bottom_range_width = 100\n  top_range_width = 200\n  bottom_range_height = 150\n  top_range_height = 200\n\nprint(bottom_range_height)\nprint(top_range_height)\nprint(bottom_range_width)\nprint(top_range_width)\n\n#middle_df = metadata_df.loc[(metadata_df['width'] >= bottom_range_width) & (metadata_df['width'] <= top_range_width) & (metadata_df['height'] >= bottom_range_height) & (metadata_df['height'] <= top_range_height)]\nmiddle_df = metadata_df\n\n\nstring_json = json.load(open(triplets_path, 'r'))\ntriplets_df = pd.DataFrame(string_json)\n\nnon_empty_triplets = triplets_df[triplets_df['Positive'] != '']\nvalid_filenames = middle_df['filename'].tolist()\n\nencoder = sklearn.preprocessing.OneHotEncoder()\none_hot_encoding = encoder.fit_transform(middle_df['cleaned_artist'].values.reshape(-1,1)).toarray()\nnum_artists = one_hot_encoding.shape[1]\n\n# No longer need the one-hot encoding since we are no longer performing artist classification\n# middle_df['artist_encoding'] = [np.array(a) for a in one_hot_encoding]\n\nmin_date = np.min(middle_df['mid-date'].values)\nmax_date = np.max(middle_df['mid-date'].values)\n\n\nnormalized_middate = (middle_df['mid-date'].values - min_date) / (max_date - min_date)\nnormalized_start_date = (middle_df['start-date'].replace('', 0).values - min_date) / (max_date - min_date)\nnormalized_end_date = (middle_df['end-date'].replace('', 0).values - min_date) / (max_date - min_date)\n\nstd_devs = np.zeros(len(normalized_start_date))\nstd_devs[normalized_start_date >= 0.0] = (normalized_middate[normalized_start_date >= 0.0] - normalized_start_date[normalized_start_date >= 0.0]) / 2.0\n\nSAME_DATE_SIGMA = 1.0\nstd_devs[normalized_start_date < 0.0] = (SAME_DATE_SIGMA / (max_date - min_date))\n\nmin_std_dev = np.min(std_devs)\nmax_std_dev = np.max(std_devs)\n\nmiddle_df['normalized_midpoint'] = normalized_middate\nmiddle_df['normalized_start'] = normalized_start_date\nmiddle_df['normalized_end'] = normalized_end_date\n\nprint(f""Number of Triplets: {len(non_empty_triplets)}"")\n\ntriplets_within_size = non_empty_triplets[(non_empty_triplets['1'].isin(valid_filenames)) & (non_empty_triplets['2'].isin(valid_filenames)) & (non_empty_triplets['anchor'].isin(valid_filenames))]\n\nprint(f""Number of Triplets: {len(triplets_within_size)}"")\n\n""""""# CREATING TRAIN / TEST SPLIT""""""\n\nfrom sklearn.model_selection import train_test_split\n\ndata_transform = transforms.Compose([\n        transforms.ToTensor()\n    ])\n\nsplit = 1 - TRAIN_TEST_RATIO\ntorch.manual_seed(0)\nnp.random.seed(0)\ntrain, test = train_test_split(triplets_within_size, test_size=split)\n\n\n## This is for training the Base VAE\ntrain_art_dataset = ArtDataset(image_folder, train, middle_df, top_range_height, top_range_width, transform=data_transform)\ntest_art_dataset = ArtDataset(image_folder, test, middle_df, top_range_height, top_range_width, transform=data_transform)\n\nbatch_sz = 12\nnum_wrkrs = 5\n\ntrain_dataset_loader = torch.utils.data.DataLoader(train_art_dataset,\n                                             batch_size=batch_sz, shuffle=True,\n                                             num_workers=num_wrkrs)\n\ntest_dataset_loader = torch.utils.data.DataLoader(test_art_dataset,\n                                             batch_size=batch_sz, shuffle=True,\n                                             num_workers=num_wrkrs)\n\nprint(f""train len: {len(train_dataset_loader)}"")\nprint(f""train len: {len(test_dataset_loader)}"")\n\n""""""# DEFINING NEURAL NETWORK ARCHITECTURE""""""\n\nimport pdb\nclass TRIPLET_VGG(nn.Module):\n    def __init__(self):\n        super(TRIPLET_VGG, self).__init__()\n        self.pretrained_net = models.vgg16(pretrained=True).to(device)\n        self.pretrained_net = self.pretrained_net.eval()\n        self.fc1 = nn.Linear(4096, 2048)\n        self.bn1 = nn.BatchNorm1d(2048)\n        self.fc2 = nn.Linear(2048, 1024)\n\n    def forward(self, x):\n      with torch.no_grad():\n        activations = None\n        output = x\n        for _, model in enumerate(self.pretrained_net.features):\n          output = model(output)\n\n        output = self.pretrained_net.avgpool(output)\n        output = output.view(output.shape[0],-1)\n        # for _, model in enumerate(pretrained_net.avgpool):\n          # output = model(output)\n        for layer_id, model in enumerate(self.pretrained_net.classifier):\n          output = model(output)\n          if layer_id == 3:\n            break\n      \n      output = self.bn1(F.relu(self.fc1(output)))\n      output = self.fc2(output)\n\n      return output\n\ntriplet_vgg = TRIPLET_VGG()\ntriplet_vgg.to(device)\nbase_optimizer = optim.Adam(triplet_vgg.parameters(), lr=1e-3)\n\n""""""# CHECKPOINT LOADING""""""\n\n################################ CHECKPOINT LOADING ##############################\ndef find_latest_checkpoint_in_dir(load_dir, prefix):\n  pretrained_models = os.listdir(load_dir)\n  if '.ipynb_checkpoints' in pretrained_models:\n    pretrained_models.remove('.ipynb_checkpoints')\n  max_model_name = """"\n  max_model_no = -1\n  for pretrained_model in pretrained_models:\n    prefix_split = pretrained_model.split(f""{prefix}"")\n    if len(prefix_split) != 2:\n      continue\n    model_no = int(prefix_split[1].split("".pt"")[0])\n    if model_no > max_model_no:\n      max_model_no = model_no\n      max_model_name = pretrained_model\n  epoch = 0\n  if max_model_name == """":\n    print(""Could not find pretrained model."")\n    return None\n  else:\n    print(f""Loaded pretrained model: {max_model_name}"")\n    epoch = max_model_no\n    return epoch, torch.load(f'{load_dir}/{max_model_name}')\n\nprint(load_model_folder)\n# Checkpoint loading for the latent artist model\nif(not find_latest_checkpoint_in_dir(load_model_folder, prefix) is None):\n  base_epoch, base_state_dict = find_latest_checkpoint_in_dir(load_model_folder, prefix)\n  epoch = base_epoch\n  triplet_vgg.load_state_dict(base_state_dict)\n#if(not find_latest_checkpoint_in_dir(LATENT_ARTIST_MODEL_LOAD_PATH, f""{DISCRIMINATOR_PREFIX}_{LATENT_PREFIX}"") is None):\n  #discrimnator_epoch, discriminator_state_dict = find_latest_checkpoint_in_dir(LATENT_ARTIST_MODEL_LOAD_PATH, f""{DISCRIMINATOR_PREFIX}_{LATENT_PREFIX}"")\n  #discriminator.load_state_dict(discriminator_state_dict)\n# Optimizers for the two networks\n# base_optimizer = optim.Adam(vae.parameters(), lr=1e-3)\n#discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=1e-3)\n\n""""""# LOSS FUNCTIONS""""""\n\n################################ HELPER METHODS ##############################\n\ndef display_first_n_of_batch(batch, n):\n  f, axarr = plt.subplots(1, n)\n  f.set_figheight(n)\n  f.set_figwidth(2 * n)\n\n  for i in range(n):\n      imgorg = np.transpose(batch[i].cpu().detach().numpy(), (1,2,0))\n      axarr[i].imshow(imgorg)\n\n  plt.show()\n\n################################ LOSS FUNCTIONS ##############################\n\ndef loss_function(recon_x, x, mean, logvariance, data_widths, data_heights):\n    recon_x = recon_x.cpu()\n    x = x.cpu()\n    mean = mean.cpu()\n    logvariance = logvariance.cpu()\n    data_widths = data_widths.cpu()\n    data_heights = data_heights.cpu()\n\n    BCE = F.binary_cross_entropy(recon_x, x, reduction='none').cpu()\n    mask_matrix = torch.zeros(recon_x.shape)\n\n    # Iterate through each element in the batch\n    for i in range(0, recon_x.shape[0]):\n      mask_matrix[i,:,0:data_heights[i],0:data_widths[i]] = torch.ones((3, data_heights[i], data_widths[i]))\n\n    # Normalize KL Divergence loss by batch size\n    KLD =  0.5 * (torch.sum(mean.pow(2) + logvariance.exp() - logvariance - 1) / recon_x.shape[0])\n\n    if MASK_TYPE == ""mask"":\n      masked_BCE = BCE * mask_matrix\n      return (KLD), (torch.sum(masked_BCE / (recon_x.shape[1] * torch.sum(data_widths * data_heights))))\n    else:\n      return (KLD), (torch.sum(BCE) / (recon_x.shape[0] * recon_x.shape[1] * recon_x.shape[2] * recon_x.shape[3]))\n\ndef discriminator_loss(prediction, gt):\n  prediction = prediction.cpu()\n  gt = gt.cpu()\n  BCE_loss = nn.BCELoss()\n  return BCE_loss(prediction, gt)\n\ndef discriminator_loss_no_reduction(prediction, gt):\n  prediction = prediction.cpu()\n  gt = gt.cpu()\n  BCE_loss = nn.BCELoss(reduction='none')\n  return BCE_loss(prediction, gt)\n\ndef discriminator_kl_divergence(mean, logvariance):\n  mean = mean.cpu()\n  logvariance = logvariance.cpu()\n\n  KLD = 0.5 * torch.sum(mean.pow(2) + logvariance.exp() - logvariance - 1)\n  return KLD\n\ndef artist_prediction_loss(artist_prediction_vector, gt_one_hot_artist, mean, logvariance):\n  artist_prediction_vector = artist_prediction_vector.cpu()\n  gt_one_hot_artist = gt_one_hot_artist.cpu()\n  mean = mean.cpu()\n  logvariance = logvariance.cpu()\n\n  BCE_loss = nn.BCELoss()\n  KLD = 0.5 * (torch.sum(mean.pow(2) + logvariance.exp() - logvariance - 1) / artist_prediction_vector.shape[0])\n  return ARTIST_PREDICTION_WEIGHT * BCE_loss(artist_prediction_vector.float(), gt_one_hot_artist.float()) + ARTIST_KLD_WEIGHT * KLD\n\ndef kl_divergence_two_gaussians(p_mean, p_logvar, q_mean, q_logvar):\n  p_mean = p_mean.cpu()\n  q_mean = q_mean.cpu()\n  p_var = p_logvar.cpu().exp()\n  q_var = q_logvar.cpu().exp()\n\n  p = torch.distributions.normal.Normal(p_mean, torch.sqrt(p_var))\n  q = torch.distributions.normal.Normal(q_mean, torch.sqrt(q_var))\n\n  return torch.distributions.kl.kl_divergence(p, q)\n\n\ndef kl_divergence_two_gaussians_std(p_mean, p_std, q_mean, q_std):\n  p_mean = p_mean.cpu()\n  q_mean = q_mean.cpu()\n  p_std = p_std.cpu()\n  q_std = q_std.cpu()\n\n  p = torch.distributions.normal.Normal(p_mean, p_std)\n  q = torch.distributions.normal.Normal(q_mean, q_std)\n\n  return torch.distributions.kl.kl_divergence(p, q)\n\ndef artist_kl_divergence(latent_mean, latent_logvar, pretrained_mean, pretrained_logvar):\n  return torch.mean(torch.sum(kl_divergence_two_gaussians(latent_mean, latent_logvar, pretrained_mean, pretrained_logvar), dim = 0))\n\ndef triplet_loss(a_mean, p_mean, n_mean):\n    mse_criterion = nn.MSELoss()\n    total_trip_loss = 0\n    num_positives = 0\n    for i in range(0, a_mean.shape[0]):\n        distance = mse_criterion(p_mean[i], a_mean[i]) - mse_criterion(n_mean[i], a_mean[i])\n        trip_loss = torch.clamp(distance + TRIPLET_ALPHA, min=0)\n        if trip_loss > torch.tensor(0):\n            num_positives += 1\n\n        total_trip_loss += trip_loss\n    if num_positives == 0:\n        return total_trip_loss.cpu()\n    else:\n        return (total_trip_loss / num_positives).cpu()\n\ndef calculate_perceptual_loss(recon_x, x):\n  # vgg_model = vgg_model.cuda()\n  recon_features = loss_network(recon_x)[2].cpu()\n  orig_features = loss_network(x.to(device))[2].cpu()\n\n  # vgg_model = vgg_model.cpu()\n  mse_criterion = nn.MSELoss()\n  return mse_criterion(recon_features, orig_features)\n\ndef calculate_artist_discriminator_loss(image_batches, triplet_latent_vectors, corresponding_latent_vectors):\n  # Here we go through all (6 choose 2) combinations of images and get the discriminator loss for each\n\n  num_same_artists = 0.0\n  num_diff_artists = 12 * triplet_latent_vectors[0].shape[0]\n\n  same_artist_disc_loss = 0.0\n  diff_artist_disc_loss = 0.0\n\n  # FIRST CASE (3 Comparisons): The images of the same author (we check if they are also the same image, and if so we mask out that loss)\n  for image_batch, triplet_latent, corr_latent in zip(image_batches, triplet_latent_vectors, corresponding_latent_vectors):\n    should_mask_vector = image_batch[""corresponding""][""should_mask""] == 0\n\n    num_same_artists += torch.sum(should_mask_vector)\n\n    latent_concat = torch.cat((triplet_latent, corr_latent), dim = 1)\n    prediction = discriminator(latent_concat)\n\n    disc_label = torch.ones(prediction.shape)\n\n    same_artist_disc_loss += torch.sum(should_mask_vector * discriminator_loss_no_reduction(prediction.squeeze(), disc_label.float().squeeze()))\n\n  # SECOND CASE (6 Comparisons): Compare each image from the triplet to each image in the corresponding that are not the same image\n  for triplet_idx, triplet_latent in enumerate(triplet_latent_vectors):\n    for corr_idx, corr_latent in enumerate(corresponding_latent_vectors):\n      if (corr_idx == triplet_idx):\n        continue\n\n    latent_concat = torch.cat((triplet_latent, corr_latent), dim = 1)\n    prediction = discriminator(latent_concat)\n\n    disc_label = torch.zeros(prediction.shape)\n    diff_artist_disc_loss += torch.sum(discriminator_loss_no_reduction(prediction.squeeze(), disc_label.float().squeeze()))\n\n  # THIRD CASE (6 Comparisons): Compare each image from the same triplet\n  latent_vectors_list = [triplet_latent_vectors, corresponding_latent_vectors]\n  for latent_vectors_triplet in latent_vectors_list:\n    for trip1_idx, trip1_latent in enumerate(latent_vectors_triplet):\n      for trip2_idx, trip2_latent in enumerate(latent_vectors_triplet):\n        if(trip1_idx == trip2_idx):\n          continue\n        latent_concat = torch.cat((trip1_latent, trip2_latent), dim = 1)\n        prediction = discriminator(latent_concat)\n\n        disc_label = torch.zeros(prediction.shape)\n        diff_artist_disc_loss += torch.sum(discriminator_loss_no_reduction(prediction.squeeze(), disc_label.float().squeeze()))\n\n\n\n  total_artist_loss = 0.0\n  if torch.sum(num_same_artists) == 0:\n    total_artist_loss = diff_artist_disc_loss / num_diff_artists\n  else:\n    total_artist_loss = (same_artist_disc_loss / num_same_artists) + (diff_artist_disc_loss / num_diff_artists)\n\n  return total_artist_loss\n\n""""""# MODEL TESTING\n\n# MODEL TRAINING\n""""""\n\ndef test_vgg_triplet(model):\n  total_loss = 0\n  with torch.no_grad():\n    for batch_idx, data in enumerate(test_dataset_loader):\n      latents = []\n      triplet_keys = ['anchor', 'positive', 'negative']\n\n      for key in triplet_keys:\n        triplet_img = data[key]['triplet']['image']\n        outputs = triplet_vgg(triplet_img.to(device))\n        latents.append(outputs)\n\n      train_loss = 0\n      base_optimizer.zero_grad()\n      latent_loss = triplet_loss(latents[0], latents[1],latents[2])\n      train_loss += latent_loss\n\n      total_loss += train_loss\n  \n  return total_loss / len(test_dataset_loader)\n\ndef train_vgg_triplet(model, start_iters, end_iters, weights_folder, prefix):\n  num_epochs = 0\n  for iters in range(start_iters, end_iters, len(train_dataset_loader)):\n    for batch_idx, data in enumerate(train_dataset_loader):\n      total_iters = batch_idx + iters\n      if total_iters >= end_iters:\n          save_filename = f'{prefix}{total_iters}.pt'\n          save_path = f'{weights_folder}/{save_filename}'\n          torch.save(vae.state_dict(), save_path)\n          return\n      print(f""Evaluating batch {total_iters} for {model_name}"")\n      end = time.time()\n      time_elapsed = end - start_time\n      if not IS_RUNNING_NOTEBOOK:\n        if time_elapsed >= TIMEOUT:\n            safe_exit(0)\n        if exit_code is not None:\n            print(""caught the exception!"")\n            save_filename = f'{prefix}{total_iters}.pt'\n            save_path = f'{weights_folder}/{save_filename}'\n            torch.save(model.state_dict(), save_path)\n            safe_exit(3)\n            return\n\n      latents = []\n      triplet_keys = ['anchor', 'positive', 'negative']\n\n      for key in triplet_keys:\n        triplet_img = data[key]['triplet']['image']\n        outputs = model(triplet_img.to(device))\n        latents.append(outputs)\n\n      train_loss = 0\n      base_optimizer.zero_grad()\n      latent_loss = triplet_loss(latents[0], latents[1],latents[2])\n      train_loss += latent_loss\n\n      train_loss.backward()\n      writer.add_scalar(f'Loss/Train_{model_name}', train_loss, total_iters)\n\n      print(f""Loss: {train_loss}"")\n\n    num_epochs += 1\n    if num_epochs % 5 == 0:\n        save_filename = f'{prefix}{total_iters}.pt'\n        save_path = f'{weights_folder}/{save_filename}'\n        torch.save(model.state_dict(), save_path)\n\n    test_loss = test_vgg_triplet(model)\n    triplet_writer.add_scalar(f'Loss/Test_{model_name}', test_loss, total_iters)\n    print(f""TEST LOSS: {test_loss} FOR EPOCH: {total_iters}"")\n\ntrain_vgg_triplet(triplet_vgg, epoch, epochs, models_folder, prefix)";triplet_vgg.py;learning-portrait-style/learning-style
bash: # -*- coding: utf-8 -*-\n\n######################## IMPORTING / GOOGLE DRIVE INTEGRATION ##############################\nimport torch\nimport argparse\nimport copy\nimport cv2\nimport json\nimport numpy as np\nimport os\nimport pandas as pd\nimport pdb\nimport pickle\nimport random\nimport scipy.ndimage\nimport shutil\nimport sklearn.preprocessing\nimport sys\n#import tensorflow as tf\nimport time\nimport torch\nimport torch.nn as nn\nimport torch.nn.functional as F\nimport torch.optim as optim\nimport torchvision\nfrom PIL import Image\nfrom itertools import product\nfrom matplotlib import pyplot as plt\nimport numpy as np\nimport pickle\nimport json\nimport pandas as pd\n#import tensorflow as tf\nfrom torchvision import datasets, transforms\nfrom torch.utils.tensorboard import SummaryWriter\nimport argparse\nimport sys\nimport pdb\nimport sklearn\nimport yaml\n\nfrom shutil import copy2\nfrom torch.autograd import Variable\nfrom torch.optim import lr_scheduler\nfrom torch.utils.data import Dataset, DataLoader\nfrom torch.utils.tensorboard import SummaryWriter\nfrom torchvision import datasets, transforms, models\nfrom yaml import load, dump\n\nfrom functools import partial\nimport signal\nimport time\nimport sys\nimport numpy as np\n\n################################ SET UP PATHS ##############################################\n# See README.md for more details\n\nmount_filepath =  # Put the path to the root directory of your project here\nmodel_name_prefix = fvae_trip_vgg\n\n################################ SETTING UP PROGRAM ARGUMENTS ##############################\n\nIS_RUNNING_NOTEBOOK = False  # ** NOTE: PLEASE DO NOT FORGET TO CHANGE THIS FLAG WHEN YOU DEPLOY **\n\nexit_code = None\n# The safe_exit function works by only registering a flag that is\n# monitored in the actual training loop\n#\n# In addition, this is the central authority on the exit code for this program \n# and will be used for the actual exit code at the end\ndef safe_exit(signal_num, ec=0, total_iters=0, exp_name = ):\n    print(Caught a signal, signal_num)\n    sys.stdout.flush()\n    global exit_code\n    exit_code = signal_num\n\nstart_time = time.time()\n\n# Catch the signal and report to safe_exit\nsignal.signal(signal.SIGTERM, safe_exit)\n\ndevice = torch.device(cuda)\n\nparser = argparse.ArgumentParser()\nparser.add_argument(yaml_config, help = No config file specified. Please specify a yaml file.)\nargs = parser.parse_args()\n\nconfig_file = open(args.yaml_config, 'r')\nconfig_args = yaml.load(config_file)\n\n\nLATENT_SIZE = config_args['LATENT_SIZE']\nLATENT_ARTIST_SIZE = config_args['LATENT_ARTIST_SIZE']\n\nPERCEP_RATIO = config_args['PERCEP_RATIO']\nKLD_RATIO = config_args['KLD_RATIO']\nTRIP_RATIO = config_args['TRIP_RATIO']\nRECON_RATIO = config_args['RECON_RATIO']\n\nOUTLIERS_REMOVED = config_args['OUTLIERS']\n\nTRAIN_TEST_RATIO = config_args['TRAIN_TEST_SPLIT']\n\nTIMEOUT = 3600\n\n\nMASK_TYPE = mask\n\n# HYPERPARAMETERS\n\n################################ HYPERPARAMETERS ##############################\n\n# Base VAE Hyperparameters\nTRIPLET_ALPHA = 1.0\n#LATENT_ARTIST_WEIGHT = 1.0 * NON_VAE_WEIGHT\nRECONSTRUCTION_WEIGHT = RECON_RATIO\nPERCEP_LOSS_WEIGHT = PERCEP_RATIO\nVAE_DIVERGENCE_WEIGHT = KLD_RATIO\nTRIPLET_LOSS_WEIGHT = TRIP_RATIO\n\n# Loss Thresholds\nMAX_VAE_LOSS_THRESHOLD = 200\nMAX_LATENT_ARTIST_LOSS_THRESHOLD = 10\n\n# Random sampling hyperparameters\nSAME_ARTIST_PROBABILITY_THRESHOLD = 0.7\n\n# Train Test Split\nTRAIN_TEST_SPLIT = TRAIN_TEST_RATIO\n\ninterval = 1 # Interval for which to print the loss\nnum_batches_per_epoch = 478\nepochs = num_batches_per_epoch * 500\nepoch = 0 # Start epoch to run from (gets automatically set in the checkpointing process)\n\n\n################################ SETTING UP DIRECTORY STRUCTURE ##############################\n\n## TODO: Update this with a better name? Let's stick with 1024 parameters and 512 for the artist latent vector\n\n# hyperparameter_name = fkld_{VAE_DIVERGENCE_WEIGHT}_percep_{PERCEP_LOSS_WEIGHT}_recon_{RECONSTRUCTION_WEIGHT}_trip_{TRIPLET_LOSS_WEIGHT}\n#hyperparameter_name = fnonvaeweight_{NON_VAE_WEIGHT}_artist_{LATENT_ARTIST_SIZE}_latent_{LATENT_SIZE}\n\nhyperparameter_name = fTRIPLET_{TRIPLET_LOSS_WEIGHT} \n\nmodel_name = f{model_name_prefix}_{hyperparameter_name} # Name of the model generated from the program arguments\n\nwriter = SummaryWriter(f'./tensorboard/{model_name}') # Tensorboard writer\n# kld_writer = SummaryWriter(f'./tensorboard/{model_name}/kld_loss') # Tensorboard writer\n# recon_writer = SummaryWriter(f'./tensorboard/{model_name}/recon_loss') # Tensorboard writer\n# artist_writer = SummaryWriter(f'./tensorboard/{model_name}/artist_loss') # Tensorboard writer\ntriplet_writer = SummaryWriter(f'./tensorboard/{model_name}/triplet_loss') # Tensorboard writer\n# perceptual_writer = SummaryWriter(f'./tensorboard/{model_name}/perceptual_loss') # Tensorboard writer\n\nprefix = padded_ # Prefix for all the files being saved\ncontinue_training = True # Specifies whether you want to continuing training from the previous epoch. Honestly don't know why you'd set it to false but you can if you want.\nmodels_folder = f{mount_filepath}/saved_models/{model_name}/models\n\nmodel_save_root = mount_filepath + '/saved_models'\nresults_folder = f{model_save_root}/{model_name}/images\nplot_folder = f{model_save_root}/{model_name}/plots\nlog_folder = f{model_save_root}/{model_name}/logs\n\nload_model_folder = f{model_save_root}/{model_name}/models\n\nos.makedirs(f{model_save_root}/{model_name}, exist_ok=True)\nos.makedirs(results_folder, exist_ok=True)\nos.makedirs(plot_folder, exist_ok=True)\n\nLATENT_PREFIX = padded_\n\n#DISCRIMINATOR_PREFIX = discriminator # DONT CHANGE THIS\n\n#LATENT_ARTIST_MODEL_NAME = f{DISCRIMINATOR_PREFIX}_{model_name_prefix}_{hyperparameter_name}\n#os.makedirs(f{model_save_root}/{LATENT_ARTIST_MODEL_NAME}, exist_ok=True)\n#os.makedirs(f{model_save_root}/{LATENT_ARTIST_MODEL_NAME}/images, exist_ok=True)\n#os.makedirs(f{model_save_root}/{LATENT_ARTIST_MODEL_NAME}/plots, exist_ok=True)\n#os.makedirs(f{model_save_root}/{LATENT_ARTIST_MODEL_NAME}/logs, exist_ok=True)\n#os.makedirs(f{model_save_root}/{LATENT_ARTIST_MODEL_NAME}/models, exist_ok=True)\n\n# If you're training the base VAE, you must still specify a load path for the latent artist network\n#LATENT_ARTIST_MODEL_LOAD_PATH = f{model_save_root}/{LATENT_ARTIST_MODEL_NAME}/models\n\n#model_name = LATENT_ARTIST_MODEL_NAME\n#prefix = LATENT_PREFIX\n#load_model_folder = LATENT_ARTIST_MODEL_LOAD_PATH\n\n# SETTING UP PATHS / DIRECTORY STRUCTURE\n\n################################ SETTING UP PATHS / DIRECTORY STRUCTURE ##############################\nmetadata_filepath = mount_filepath + 'FINAL_DATASET_SIZES.csv'\nimage_zip_folder = mount_filepath + 'Images.zip'\nimage_folder = mount_filepath + Images\n# triplets_path = mount_filepath + Triplets/4700_labels_copy.json \ntriplets_path = mount_filepath + Triplets/6384_labels.json\n\nwriter = SummaryWriter(f'./tensorboard/{model_name}') # Tensorboard writer\nprefix = padded_ # Prefix for all the files being savedc\nmodels_folder = f{mount_filepath}/saved_models/{model_name}/models\n\nmodel_save_root = mount_filepath + '/saved_models'\nresults_folder = f{model_save_root}/{model_name}/images\nlog_folder = f{model_save_root}/{model_name}/logs\n\nos.makedirs(f{model_save_root}/{model_name}, exist_ok=True)\nos.makedirs(results_folder, exist_ok=True)\nos.makedirs(log_folder, exist_ok=True)\nos.makedirs(f{model_save_root}/{model_name}/models, exist_ok=True)\n\n# ART DATASET CLASS\n\n################################ ART DATASET CLASS ##############################\n\nclass ArtDataset(Dataset):\n    def __init__(self, image_folder, triplet_df, metadata_df, largest_height, largest_width, transform=None):\n        \n        Args:\n            image_folder (string): Directory with all the images.\n            metadata_df (Pandas Dataframe): Dataframe containing the metadata\n            transform (Torchvision.Transform) Transform to be applied to the image data\n        \n        self.transform = transform\n        self.metadata = metadata_df.reset_index()\n        self.triplets = triplet_df\n        self.image_folder = image_folder\n        self.largest_height = largest_height\n        self.largest_width = largest_width\n\n    def __len__(self):\n        return len(self.triplets.index)\n\n    def return_transformed_image(self, image_filename):\n        image = Image.open(os.path.join(self.image_folder, image_filename))\n        (width, height) = image.size\n        tensor_image = self.transform(image)\n        pad_layer = nn.ZeroPad2d((0, self.largest_width - width, 0, self.largest_height - height))\n        return pad_layer(tensor_image)\n\n    def get_random_artist_image(self, artist):\n        artist_indices = self.metadata[self.metadata['cleaned_artist'] == artist].index\n        random_artist_idx = random.choice(artist_indices)\n        return self.metadata.iloc[random_artist_idx]\n\n    def __getitem__(self, idx):\n      row = self.triplets.iloc[idx]\n\n      positive_idx = int(row['Positive']) - 1\n      negative_idx = (1 - positive_idx)\n\n      triplet_names = ['1', '2', 'anchor']\n\n      # Re-Arranges it so now it's of the form positive, negative, anchor.\n      triplet_files = [row[triplet_names[positive_idx]], row[triplet_names[negative_idx]], row[triplet_names[2]]]\n      labels = ['positive', 'negative', 'anchor']\n\n      triplet_objects = {}\n\n      for filename, label in zip(triplet_files, labels):\n        triplet_object = {}\n\n        triplet_image = {}\n        triplet_corresponding = {}\n\n        image_metadata = self.metadata.loc[self.metadata['filename'] == filename]\n        image_name = image_metadata['filename'].values[0]\n\n        triplet_image['artist'] = image_metadata['cleaned_artist'].values[0]\n        triplet_image['image'] = self.return_transformed_image(image_name)\n\n        triplet_image['normalized_midpoint'] = image_metadata['normalized_midpoint'].values[0]\n        triplet_image['normalized_start'] = image_metadata['normalized_start'].values[0]\n        triplet_image['normalized_end'] = image_metadata['normalized_end'].values[0]\n\n        triplet_image['width'] = image_metadata['width'].values[0]\n        triplet_image['height'] = image_metadata['height'].values[0]\n\n        #random_artist = self.get_random_artist_image(triplet_image['artist'])\n\n        #triplet_corresponding['image'] = self.return_transformed_image(random_artist['filename'])\n        #triplet_corresponding['filename'] = random_artist['filename']\n\n        #triplet_corresponding['normalized_midpoint'] = random_artist['normalized_midpoint']\n        #triplet_corresponding['normalized_start'] = random_artist['normalized_start']\n        #triplet_corresponding['normalized_end'] = random_artist['normalized_end']\n\n        #triplet_corresponding['width'] = random_artist['width']\n        #triplet_corresponding['height'] = random_artist['height']\n\n        #triplet_corresponding['should_mask'] = (random_artist['filename'] == image_metadata['filename'].values[0])\n\n        triplet_object['triplet'] = triplet_image\n        #triplet_object['corresponding'] = triplet_corresponding\n\n        triplet_objects[label] = triplet_object\n\n      return triplet_objects\n\n# READING IN DATA FROM CSV / PREPROCESSING\n\n################################ READING IN DATA FROM CSV / PREPROCESSING ##############################\n\nimport json\nimport pandas as pd\n#pdb.set_trace()\n\nmetadata_filepath = mount_filepath + 'FINAL_DATASET_SIZES.csv'\nimage_zip_folder = mount_filepath + 'Images.zip'\n\nimage_folder = mount_filepath + Images\n\nmetadata_df = pd.read_csv(metadata_filepath)\nprint(metadata_df.shape)\nmetadata_df = metadata_df.fillna()\nartists_column = metadata_df['cleaned_artist'].str.lower()\nknown_artists = metadata_df[(artists_column != 'unsure') & (artists_column != 'anonymous')]\n\nanonymous_artists = metadata_df[(artists_column == 'unsure') ^ (artists_column == 'anonymous')]\n\n# To use all the data just set min_thresh = 0 and max_thresh = 1\nmin_thresh = 0.00\nmax_thresh = 1.00\n\n# Only keep the artists that are known\n# CURRENTLY COMMENTED SINCE WE DON'T CARE IF THE ARTISTS ARE UNKOWN!\n# metadata_df = known_artists\n\nsorted_widths = sorted(metadata_df['width'].tolist())\nsorted_heights = sorted(metadata_df['height'].tolist())\n\nif OUTLIERS_REMOVED == outliers:\n  bottom_range_height = sorted_heights[max(0, int(len(sorted_heights) * min_thresh) - 1)]\n  top_range_height = sorted_heights[max(0, int(len(sorted_heights) * max_thresh) - 1)] + 1\n  bottom_range_width = sorted_widths[max(0, int(len(sorted_widths) * min_thresh) - 1)]\n  top_range_width = sorted_widths[max(0, int(len(sorted_widths) * max_thresh) - 1)]\nelse:\n  bottom_range_width = 100\n  top_range_width = 200\n  bottom_range_height = 150\n  top_range_height = 200\n\nprint(bottom_range_height)\nprint(top_range_height)\nprint(bottom_range_width)\nprint(top_range_width)\n\n#middle_df = metadata_df.loc[(metadata_df['width'] >= bottom_range_width) & (metadata_df['width'] <= top_range_width) & (metadata_df['height'] >= bottom_range_height) & (metadata_df['height'] <= top_range_height)]\nmiddle_df = metadata_df\n\n\nstring_json = json.load(open(triplets_path, 'r'))\ntriplets_df = pd.DataFrame(string_json)\n\nnon_empty_triplets = triplets_df[triplets_df['Positive'] != '']\nvalid_filenames = middle_df['filename'].tolist()\n\nencoder = sklearn.preprocessing.OneHotEncoder()\none_hot_encoding = encoder.fit_transform(middle_df['cleaned_artist'].values.reshape(-1,1)).toarray()\nnum_artists = one_hot_encoding.shape[1]\n\n# No longer need the one-hot encoding since we are no longer performing artist classification\n# middle_df['artist_encoding'] = [np.array(a) for a in one_hot_encoding]\n\nmin_date = np.min(middle_df['mid-date'].values)\nmax_date = np.max(middle_df['mid-date'].values)\n\n\nnormalized_middate = (middle_df['mid-date'].values - min_date) / (max_date - min_date)\nnormalized_start_date = (middle_df['start-date'].replace('', 0).values - min_date) / (max_date - min_date)\nnormalized_end_date = (middle_df['end-date'].replace('', 0).values - min_date) / (max_date - min_date)\n\nstd_devs = np.zeros(len(normalized_start_date))\nstd_devs[normalized_start_date >= 0.0] = (normalized_middate[normalized_start_date >= 0.0] - normalized_start_date[normalized_start_date >= 0.0]) / 2.0\n\nSAME_DATE_SIGMA = 1.0\nstd_devs[normalized_start_date < 0.0] = (SAME_DATE_SIGMA / (max_date - min_date))\n\nmin_std_dev = np.min(std_devs)\nmax_std_dev = np.max(std_devs)\n\nmiddle_df['normalized_midpoint'] = normalized_middate\nmiddle_df['normalized_start'] = normalized_start_date\nmiddle_df['normalized_end'] = normalized_end_date\n\nprint(fNumber of Triplets: {len(non_empty_triplets)})\n\ntriplets_within_size = non_empty_triplets[(non_empty_triplets['1'].isin(valid_filenames)) & (non_empty_triplets['2'].isin(valid_filenames)) & (non_empty_triplets['anchor'].isin(valid_filenames))]\n\nprint(fNumber of Triplets: {len(triplets_within_size)})\n\n# CREATING TRAIN / TEST SPLIT\n\nfrom sklearn.model_selection import train_test_split\n\ndata_transform = transforms.Compose([\n        transforms.ToTensor()\n    ])\n\nsplit = 1 - TRAIN_TEST_RATIO\ntorch.manual_seed(0)\nnp.random.seed(0)\ntrain, test = train_test_split(triplets_within_size, test_size=split)\n\n\n## This is for training the Base VAE\ntrain_art_dataset = ArtDataset(image_folder, train, middle_df, top_range_height, top_range_width, transform=data_transform)\ntest_art_dataset = ArtDataset(image_folder, test, middle_df, top_range_height, top_range_width, transform=data_transform)\n\nbatch_sz = 12\nnum_wrkrs = 5\n\ntrain_dataset_loader = torch.utils.data.DataLoader(train_art_dataset,\n                                             batch_size=batch_sz, shuffle=True,\n                                             num_workers=num_wrkrs)\n\ntest_dataset_loader = torch.utils.data.DataLoader(test_art_dataset,\n                                             batch_size=batch_sz, shuffle=True,\n                                             num_workers=num_wrkrs)\n\nprint(ftrain len: {len(train_dataset_loader)})\nprint(ftrain len: {len(test_dataset_loader)})\n\n# DEFINING NEURAL NETWORK ARCHITECTURE\n\nimport pdb\nclass TRIPLET_VGG(nn.Module):\n    def __init__(self):\n        super(TRIPLET_VGG, self).__init__()\n        self.pretrained_net = models.vgg16(pretrained=True).to(device)\n        self.pretrained_net = self.pretrained_net.eval()\n        self.fc1 = nn.Linear(4096, 2048)\n        self.bn1 = nn.BatchNorm1d(2048)\n        self.fc2 = nn.Linear(2048, 1024)\n\n    def forward(self, x):\n      with torch.no_grad():\n        activations = None\n        output = x\n        for _, model in enumerate(self.pretrained_net.features):\n          output = model(output)\n\n        output = self.pretrained_net.avgpool(output)\n        output = output.view(output.shape[0],-1)\n        # for _, model in enumerate(pretrained_net.avgpool):\n          # output = model(output)\n        for layer_id, model in enumerate(self.pretrained_net.classifier):\n          output = model(output)\n          if layer_id == 3:\n            break\n      \n      output = self.bn1(F.relu(self.fc1(output)))\n      output = self.fc2(output)\n\n      return output\n\ntriplet_vgg = TRIPLET_VGG()\ntriplet_vgg.to(device)\nbase_optimizer = optim.Adam(triplet_vgg.parameters(), lr=1e-3)\n\n# CHECKPOINT LOADING\n\n################################ CHECKPOINT LOADING ##############################\ndef find_latest_checkpoint_in_dir(load_dir, prefix):\n  pretrained_models = os.listdir(load_dir)\n  if '.ipynb_checkpoints' in pretrained_models:\n    pretrained_models.remove('.ipynb_checkpoints')\n  max_model_name = \n  max_model_no = -1\n  for pretrained_model in pretrained_models:\n    prefix_split = pretrained_model.split(f{prefix})\n    if len(prefix_split) != 2:\n      continue\n    model_no = int(prefix_split[1].split(.pt)[0])\n    if model_no > max_model_no:\n      max_model_no = model_no\n      max_model_name = pretrained_model\n  epoch = 0\n  if max_model_name == :\n    print(Could not find pretrained model.)\n    return None\n  else:\n    print(fLoaded pretrained model: {max_model_name})\n    epoch = max_model_no\n    return epoch, torch.load(f'{load_dir}/{max_model_name}')\n\nprint(load_model_folder)\n# Checkpoint loading for the latent artist model\nif(not find_latest_checkpoint_in_dir(load_model_folder, prefix) is None):\n  base_epoch, base_state_dict = find_latest_checkpoint_in_dir(load_model_folder, prefix)\n  epoch = base_epoch\n  triplet_vgg.load_state_dict(base_state_dict)\n#if(not find_latest_checkpoint_in_dir(LATENT_ARTIST_MODEL_LOAD_PATH, f{DISCRIMINATOR_PREFIX}_{LATENT_PREFIX}) is None):\n  #discrimnator_epoch, discriminator_state_dict = find_latest_checkpoint_in_dir(LATENT_ARTIST_MODEL_LOAD_PATH, f{DISCRIMINATOR_PREFIX}_{LATENT_PREFIX})\n  #discriminator.load_state_dict(discriminator_state_dict)\n# Optimizers for the two networks\n# base_optimizer = optim.Adam(vae.parameters(), lr=1e-3)\n#discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=1e-3)\n\n# LOSS FUNCTIONS\n\n################################ HELPER METHODS ##############################\n\ndef display_first_n_of_batch(batch, n):\n  f, axarr = plt.subplots(1, n)\n  f.set_figheight(n)\n  f.set_figwidth(2 * n)\n\n  for i in range(n):\n      imgorg = np.transpose(batch[i].cpu().detach().numpy(), (1,2,0))\n      axarr[i].imshow(imgorg)\n\n  plt.show()\n\n################################ LOSS FUNCTIONS ##############################\n\ndef loss_function(recon_x, x, mean, logvariance, data_widths, data_heights):\n    recon_x = recon_x.cpu()\n    x = x.cpu()\n    mean = mean.cpu()\n    logvariance = logvariance.cpu()\n    data_widths = data_widths.cpu()\n    data_heights = data_heights.cpu()\n\n    BCE = F.binary_cross_entropy(recon_x, x, reduction='none').cpu()\n    mask_matrix = torch.zeros(recon_x.shape)\n\n    # Iterate through each element in the batch\n    for i in range(0, recon_x.shape[0]):\n      mask_matrix[i,:,0:data_heights[i],0:data_widths[i]] = torch.ones((3, data_heights[i], data_widths[i]))\n\n    # Normalize KL Divergence loss by batch size\n    KLD =  0.5 * (torch.sum(mean.pow(2) + logvariance.exp() - logvariance - 1) / recon_x.shape[0])\n\n    if MASK_TYPE == mask:\n      masked_BCE = BCE * mask_matrix\n      return (KLD), (torch.sum(masked_BCE / (recon_x.shape[1] * torch.sum(data_widths * data_heights))))\n    else:\n      return (KLD), (torch.sum(BCE) / (recon_x.shape[0] * recon_x.shape[1] * recon_x.shape[2] * recon_x.shape[3]))\n\ndef discriminator_loss(prediction, gt):\n  prediction = prediction.cpu()\n  gt = gt.cpu()\n  BCE_loss = nn.BCELoss()\n  return BCE_loss(prediction, gt)\n\ndef discriminator_loss_no_reduction(prediction, gt):\n  prediction = prediction.cpu()\n  gt = gt.cpu()\n  BCE_loss = nn.BCELoss(reduction='none')\n  return BCE_loss(prediction, gt)\n\ndef discriminator_kl_divergence(mean, logvariance):\n  mean = mean.cpu()\n  logvarianc""""""

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-44-13848e62a2a8>, line 2)

In [ ]:
len(python_files_df)

In [ ]:
repo = "CaitinZhao/cvpr2019_Pyramid-Feature-Attention-Network-for-Saliency-detection"

In [ ]:
python_files_df[python_files_df['content'].str.contains('~')].sum()

In [ ]:
python_files_df[python_files_df['repo_name'] == repo].iloc[3]['content']

In [ ]:
python_files_df.head()

In [ ]:
repo_files_df = pd.read_csv('data/scraped_python_files.csv')

In [ ]:
repo_files_df['repo_name'].unique()

In [ ]:
repo_files.to_csv('data/scraped_python_files.csv', index=False)

In [ ]:
repo_files_df['repo_name']

In [ ]:
import_corpus_df = pd.read_csv('output/module_corpus.csv')#['repo'].unique().shape

In [ ]:
python_files_df = repo_files_df

In [ ]:
import ast

In [ ]:
repo_names = python_files_df['repo_name']
paperswithcode_df, all_papers_df = paperswithcode_tasks.get_paperswithcode_dfs()
papers_with_repo_df = paperswithcode_tasks.get_papers_with_repo_df(all_papers_df, paperswithcode_df, repo_names)
papers_with_repo_df = paperswithcode_tasks.get_papers_with_biggest_tasks(papers_with_repo_df, 500)

In [ ]:
import_corpus_df['imports'] = import_corpus_df['imports'].apply(ast.literal_eval)
per_repo_imports = import_corpus_df.groupby('repo')['imports'].agg(sum).apply(set)

In [ ]:
per_repo_imports.shape

In [ ]:
paperswithcode_with_imports_df = get_paperswithcode_with_imports_df(papers_with_repo_df, per_repo_imports)

In [ ]:
ps = list(pathlib.Path("repos/lambdaofgod/examples-counterexamples").rglob("*"))

In [ ]:
%cd ..